## Application of deep learning method in automatically detecting rainfall-induced shallow landslides in a data-sparse context
Roquia Salam, Filiberto Pla Bañón, Bayes Ahmed, Marco Painho




## Mount Google Drive and install necessary libraries

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install segmentation_models rasterio geopandas contextily # This line will install the packages/libraries which are not present in Google Colab.

In [ ]:
!pip install -U -q segmentation-models  # you may need to adjust according to your device configuration
!pip install -q tensorflow==2.2.1
!pip install -q keras==2.5
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow import keras
import pandas as pd
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import sys
sys.path.append('/content/drive/MyDrive/Data for the experiment/Fold 4') # mention your folder in the google drive where your data is stored


physical_devices = tf.config.experimental.list_physical_devices('GPU')

for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

if tf.test.gpu_device_name():

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


## Load the NumPy arrays which contains the training, validation, and testing data



In [ ]:
X_train = np.load(f'/content/drive/MyDrive/Data for the experiment/Fold 4/Xtrain.npy')  #mention the path of the data
Y_train = np.load(f'/content/drive/MyDrive/Data for the experiment/Fold 4/Ytrain.npy')
X_val = np.load(f'/content/drive/MyDrive/Data for the experiment/Fold 4/Xval.npy')
Y_val = np.load(f'/content/drive/MyDrive/Data for the experiment/Fold 4/Yval.npy')
X_test = np.load(f'/content/drive/MyDrive/Data for the experiment/Fold 4/Xtest.npy')
Y_test = np.load(f'/content/drive/MyDrive/Data for the experiment/Fold 4/Ytest.npy')


print("Shape of the training dataset (satellite image): ", X_train.shape)
print("Shape of the training dataset (label image): ",Y_train.shape)
print("Shape of the validation dataset (satellite image): ",X_val.shape)
print("Shape of the validation dataset (label image): ",Y_val.shape)
print("Shape of the testing dataset (satellite image): ",X_test.shape)
print("Shape of the testing dataset (label image): ",Y_test.shape)

Shape of the training dataset (satellite image):  (1000, 64, 64, 4)
Shape of the training dataset (label image):  (1000, 64, 64, 1)
Shape of the validation dataset (satellite image):  (500, 64, 64, 4)
Shape of the validation dataset (label image):  (500, 64, 64, 1)
Shape of the testing dataset (satellite image):  (500, 64, 64, 4)
Shape of the testing dataset (label image):  (500, 64, 64, 1)


In [ ]:
# Visualise some data
for i in range(5):
    f, axarr = plt.subplots(1,2,figsize=(8,8))
    axarr[0].imshow(X_train[i][:,:,:3])
    axarr[0].set_title("Satellite Image")
    axarr[1].imshow(np.squeeze(Y_train[i]))
    axarr[1].set_title("Label/Ground Truth Image")

# **INITIALIZE MODEL TRAINING PARAMETERS**

In [ ]:
# Here define the evaluation metrics - Precision, Recall, FScore, IoU
metrics = [sm.metrics.Precision(threshold=0.5),sm.metrics.Recall(threshold=0.5),sm.metrics.FScore(threshold=0.5,beta=1),sm.metrics.IOUScore(threshold=0.5)]

In [ ]:
# Loss function
import tensorflow.keras.backend as K

smooth = 1

def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dsc(y_true, y_pred)
    return loss

# **U-Net Segmentation Model**



In [ ]:
# Model training - Results are saved in a .csv file

# Size of the tiles/patches
size = X_train.shape[2] # This line takes the value of the 3rd index which in this is taken from X_train shape = 1000, 64, 64, 4, that is 64.

# Image bands
img_bands = X_train.shape[3] # This line takes the value of the 4th index which in this is taken from X_train shape = 1000, 64, 64, 4, that is 4.

# Loss function.
loss=dice_loss

# Number of filters. We set a range of the number of filters for the convolutional layers.
filters = [4, 8,16,32,64]

# The learning rate is a tuning parameter in an optimization algorithm that determines the step size
# at each iteration while moving toward a minimum of a loss function.

lr = [10e-3, 5e-4, 10e-4, 5e-5, 10e-5]

# Batch sizes. This considers how many patches the model will take during the training phases. A value of 4 means 4 patches of images (from 1000) will be taken as a batch while training simultaneously.
batch_size = [4, 8, 16, 32]

# Epochs. The number of iterations the model will train.
epochs = 100 #try with different epochs from 5-100/200 based on the size of your training samples

# Dictionary that will save the results.
dic = {}

# Hyperparameters. These are the keys where the associated information for each hyperparameter will be saved.
dic["model"] = [] # Name of the model
dic["batch_size"] = [] # Batch Size
dic["learning_rate"] = [] # Learning rate
dic["filters"] = [] # Number of filters

# Metrics on the test set. It will save the metrics after evaluating the model on the test set.
dic["precision_area"] = []
dic["recall_area"] = []
dic["f1_score_area"] = []
dic["IOU_Score"] = []
dic["accuracy"] = []
dic["loss"] = []

## Here a nested for-loop performed to check all possible hyperparameter combinations that we set above.

######################################## List to store results during the loop
results_list = []

# loop over all the filters in the filter list
for fiilter in filters:
    # loop over the learning rates
    for learning_rate in lr:
        # loop over all batch sizes in batch_size list
        for batch in batch_size:
            print('_______________________________________________________________________________')
            print('Filters: ', fiilter)
            print('Learning rate: ', learning_rate)
            print('Batch size: ', batch)

            # define the model architecture here.
            def unet(lr,filtersFirstLayer, pretrained_weights = None,input_size = (size,size,img_bands)):
                inputs = Input(input_size)
                conv1 = Conv2D(filtersFirstLayer, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(inputs)
                conv1 = Conv2D(filtersFirstLayer, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv1)
                pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
                conv2 = Conv2D(filtersFirstLayer*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(pool1)
                conv2 = Conv2D(filtersFirstLayer*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv2)
                pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
                conv3 = Conv2D(filtersFirstLayer*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(pool2)
                conv3 = Conv2D(filtersFirstLayer*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv3)
                pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
                conv4 = Conv2D(filtersFirstLayer*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(pool3)
                conv4 = Conv2D(filtersFirstLayer*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv4)
                pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

                conv5 = Conv2D(filtersFirstLayer*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(pool4)
                conv5 = Conv2D(filtersFirstLayer*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv5)

                up6 = Conv2D(filtersFirstLayer*8, 2, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(UpSampling2D(size = (2,2))(conv5))
                merge6 = concatenate([conv4,up6], axis = 3)
                conv6 = Conv2D(filtersFirstLayer*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(merge6)
                conv6 = Conv2D(filtersFirstLayer*8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv6)

                up7 = Conv2D(filtersFirstLayer*4, 2, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(UpSampling2D(size = (2,2))(conv6))
                merge7 = concatenate([conv3,up7], axis = 3)
                conv7 = Conv2D(filtersFirstLayer*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(merge7)
                conv7 = Conv2D(filtersFirstLayer*4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv7)

                up8 = Conv2D(filtersFirstLayer*2, 2, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(UpSampling2D(size = (2,2))(conv7))
                merge8 = concatenate([conv2,up8], axis = 3)
                conv8 = Conv2D(filtersFirstLayer*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(merge8)
                conv8 = Conv2D(filtersFirstLayer*2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv8)

                up9 = Conv2D(filtersFirstLayer, 2, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(UpSampling2D(size = (2,2))(conv8))
                merge9 = concatenate([conv1,up9], axis = 3)
                conv9 = Conv2D(filtersFirstLayer, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(merge9)
                conv9 = Conv2D(filtersFirstLayer, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv9)
                conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'glorot_normal')(conv9)
                conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

                model = Model(inputs, conv10)

                model.compile(optimizer=Adam(learning_rate=lr), loss=loss, metrics=[metrics,'accuracy'])

                #model.summary()

                if(pretrained_weights):
                    model.load_weights(pretrained_weights)

                return model

            # Load the model in a new variable called "model".
            model = unet(filtersFirstLayer= fiilter, lr = learning_rate, input_size = (size,size,img_bands))

            # Stop the training if the validation loss does not decrease after 30 epochs. This is done to avoid over-fitting of the model.
            early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss',
                              patience = 30, # how many epochs to continue running the model after not seeing any change in the "val_loss"/you can change it
                              restore_best_weights = True) # update the model weights

            # Save the models only when validation loss decrease
            model_checkpoint = tf.keras.callbacks.ModelCheckpoint(f'/content/drive/MyDrive/Data for the experiment/Results/weights/unet_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}.hdf5',
                                                                  monitor='val_loss', mode='min',verbose=0, save_best_only=True,save_weights_only = True)
            #here, the weights will be saved in the following folder in the google drive: /content/drive/MyDrive/Data for the experiment/Results/weights.../you have to mention

            # fit the model with 25% of the dataset used as the validation set
            history = model.fit(X_train,Y_train,
                                validation_data=(X_val, Y_val), #validation_split=0.2...if you use the validation data from the train set
                                batch_size = batch,epochs=epochs,
                                callbacks = [model_checkpoint, early_stop], verbose=1)

            # Data augmentation: define geometric transformation of original training images//If you want to apply the data augmentation techniques,
            #uncomment the following lines of code and comment the previous lines of code to fit model 

            #data_aumentation_flag = 1

            #train_generator = ImageDataGenerator(
                #featurewise_center=True,
                #featurewise_std_normalization=True,
                #width_shift_range=0.2,
                #height_shift_range=0.2,
                #rotation_range=20,
                #zoom_range=[1.0,1.2],
                #horizontal_flip=True)

            #train_generator.fit(X_train)

            #val_generator = ImageDataGenerator(
                #featurewise_center=True,
                #featurewise_std_normalization=True)

            #val_generator.fit(X_val)

            #history = model.fit(train_generator.flow(X_train, Y_train, batch_size=batch),
                        #steps_per_epoch = X_train.shape[0] / batch,
                        #epochs = epochs,
                        #validation_data=val_generator.flow(X_val, Y_val),
                        #callbacks = [model_checkpoint, early_stop],
                        #verbose=1 )

            # summarize history for f1-score
            plt.plot(history.history['f1-score'])
            plt.plot(history.history['val_f1-score'])
            plt.title('model f1-score')
            plt.ylabel('f1-score')
            plt.xlabel('epoch')
            plt.legend(['train', 'validation'], loc='upper left')
            # save plots locally
            plt.savefig(f"/content/drive/MyDrive/Data for the experiment/Results/plots/unet_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}_f1_score.png")
            #here, the plots will be saved in the following folder in the google drive: /content/drive/MyDrive/Data for the experiment/Results/plots..
            plt.show()
            # summarize history for loss
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('model loss')
            plt.ylabel('loss')
            plt.xlabel('epoch')
            plt.legend(['train', 'validation'], loc='upper left')
            plt.savefig(f"/content/drive/MyDrive/Data for the experiment/Results/plots/unet_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}_val_loss.png")
            plt.show()

            # load unet to evaluate the test data
            load_unet = unet(filtersFirstLayer= fiilter, lr = learning_rate,input_size=(size,size,img_bands))
            # load the last saved weight from the training
            load_unet.load_weights(f"/content/drive/MyDrive/Data for the experiment/Results/weights/net_size_{size}_filters_{fiilter}_batch_size_{batch}_lr_{learning_rate}.hdf5")

           # Here, evaluate the model performance on the test set, a set of data that the model has never seen before and therefore remains an objective tool to test the performance of the model.
            res_1= load_unet.evaluate(X_test,Y_test)

          ########If you want to save the performance metrics for the train and validation data in a csv file
            res_2= load_unet.evaluate(X_train,Y_train)
            res_3= load_unet.evaluate(X_val,Y_val)


            # save results of the test data on the dictionary and then output them all in a Excel CSV file.
            dic["model"].append("Unet")
            dic["batch_size"].append(batch)
            dic["learning_rate"].append(learning_rate)
            dic["filters"].append(fiilter)
            dic["loss"].append(res_1[0])
            dic["precision_area"].append(res_1[1])
            dic["recall_area"].append(res_1[2])
            dic["f1_score_area"].append(res_1[3])
            dic["IOU_Score"].append(res_1[4])
            dic["accuracy"].append(res_1[5])


            # Convert results to a dataframe
            results = pd.DataFrame(dic)
            # Export as csv
            results.to_csv(f'/content/drive/MyDrive/Data for the experiment/Results/csv/results_Unet.csv', index = False)
            # here, the csv file will be saved in the following folder in the google drive: /content/drive/MyDrive/Data for the experiment/Results/csv..
            
            # Save results in the list for the train and validation datsets
            results_list.append({
               "model": "Unet",
               "batch_size": batch,
               "learning_rate": learning_rate,
               "filters": fiilter,
               "training_loss_train": res_2[0],
               "training_precision_area_train": res_2[1],
               "training_recall_area_train": res_2[2],
               "training_f1_score_area_train": res_2[3],
               "training_IOU_Score_train": res_2[4],
               "training_accuracy_train": res_2[5],
               "validation_loss_val": res_3[0],
               "validation_precision_area_val": res_3[1],
               "validation_recall_area_val": res_3[2],
               "validation_f1_score_area_val": res_3[3],
               "validation_IOU_Score_val": res_3[4],
               "validation_accuracy_val": res_3[5]
            })
           # Convert results_list to a DataFrame
            results_1 = pd.DataFrame(results_list)

           # Export as csv for the train and validation data
            results_1.to_csv(f'/content/drive/MyDrive/Data for the experiment/Results/csv/results_1_Unet.csv', index=False)


# **PREDICT LANDSLIDES ON THE TEST SET**

In [ ]:
# Load the best model based on the best performances on the test set (check CSV file)
# Loading the model weights
unet_best = unet(filtersFirstLayer= 8,lr = 0.001,input_size=(size,size,img_bands)) #  use the same number of filters and learning rate of the chosen model to initialise the network.
unet_best.load_weights("/content/drive/MyDrive/Data for the experiment/Results/weights/unet_size_64_filters_8_batch_size_8_lr_0.001.hdf5")

no = 100   #X_test.shape[0]
# Plot predictions on test set
for i in range(no):
    preds_train_1 = unet_best.predict(np.expand_dims(X_test[i],axis = 0), verbose=0)
    # It's possible to change the 0.5 threshold to improve the results;
    preds_train_t1 = (preds_train_1 > 0.5).astype(np.uint8)
    f, axarr = plt.subplots(1,3,figsize=(10,10))
    axarr[0].imshow(X_test[i][:,:,:3])
    axarr[0].set_title("Satellite image")
    axarr[1].imshow(np.squeeze(preds_train_t1))
    axarr[1].set_title("Predictions made by the model")
    axarr[2].imshow(np.squeeze(Y_test[i]))
    axarr[2].set_title("Label/Ground truth image")